In [ ]:
print("Um cuspidor esta nascendo!")

Um cuspidor esta nascendo!


In [1]:
!pip install gradio duckdb pandas --quiet


In [109]:
#csv operations
import chardet
import pandas as pd
import csv

class CSVOperator():
    def __init__(self, file):
        self.df = pd.DataFrame()
        self.primary_key = ""
        self.cols = []
        if file is not None:
            self.load_bronze(file)
        else:
          return "Envie um file CSV."

    def get_file_options(self, filename):
        options = {}
        with open(filename, 'rb') as f:
            sample = f.read(10000)
            sniffer = csv.Sniffer()
            dialect = sniffer.sniff(sample)
            options["delimiter"] = dialect.delimiter
            options["encoding"] = chardet.detect(sample)['encoding']
        return options

    def preview_bronze(self):
        return self.df.head(5)

    def set_primary_key(self, column_name):
        if column_name in self.df.columns:
            self.primary_key = column_name

    def load_bronze(self,file):
        try:
          #op = self.get_file_options(file.name)
          self.df = pd.read_csv(file.name,
                               # delimiter=op["encoding"],
                                #encoding=op["encoding"],
                                skipinitialspace=True,
                                quotechar='"'
                                )
          self.cols = list(self.df.columns)



          #return self.df
        except Exception as e:
          return str(e)



In [108]:

url = 'https://raw.githubusercontent.com/alura-cursos/llamaIndex_pandas_query/refs/heads/main/Dados/vendas.csv'
df = pd.read_csv(url)
list(df.columns)

['ID_compra',
 'filial',
 'cidade',
 'tipo_cliente',
 'genero',
 'tipo_produto',
 'preco_unitario',
 'quantidade',
 'imposto_5%',
 'total',
 'data',
 'hora',
 'forma_pagamento',
 'avaliacao']

In [ ]:
import gradio as gr

def run_bronze(file):
    csv_operator = CSVOperator(file)
    return csv_operator

def preview_bronze(csv_operator):
    return csv_operator.preview_bronze()

def set_pk_column(csv_operator, pk_column):
    if csv_operator is not None:
        csv_operator.set_primary_key(pk_column)
    return csv_operator

#button options

def toggle_button_pk(csv_operator,checkbox_value):
    if checkbox_value:
        return gr.update(interactive=True, visible=True,choices=csv_operator.cols)
    else:
        return gr.update(interactive=False, visible=False)

with gr.Blocks(title="Arquitetura Medalhão - Data Spitter - CSV - multi Reads") as demo:
    gr.Markdown("#Arquitetura Medalhão (Bronze / Silver / Gold)")
    gr.Markdown("# Spitter Data Lakehouse Explorer")
    gr.Markdown("Leia seus dados, configure as operacoes, e maos a obra. Apresentamos suporte a cargas incrementais e de sobrescrita")


    #BronzeLayer
    with gr.Tab("Bronze Layer"):
        gr.Markdown("### Upload dos dados brutos")
        state_csv_operator = gr.State(None)
        state_choices = gr.State([])
        csv_bronze = gr.File(label="Selecione um CSV",file_count="single", type="filepath")
        csv_bronze.upload(run_bronze, inputs=csv_bronze, outputs=state_csv_operator)

        gr.Markdown("### Preview dos dados brutos")
        bronze_btn = gr.Button("Preview Bronze")
        bronze_table = gr.DataFrame(label="Bronze Data")
        bronze_btn.click(preview_bronze, inputs=state_csv_operator, outputs=bronze_table)

        pk_checkbox = gr.Checkbox(label="Does the table have a primary key??", value=False)
        pk_dropdown = gr.Dropdown(label="Select the primary key", choices=[], interactive=False, visible=False)
        pk_checkbox.change(toggle_button_pk, inputs=[state_csv_operator, pk_checkbox], outputs=pk_dropdown)
        pk_dropdown.change(set_pk_column, inputs=[state_csv_operator, pk_dropdown], outputs=state_csv_operator)







    # Upload atualiza apenas o state

    # # Camada Silver
    # with gr.Tab("Camada Silver"):
    #     gr.Markdown("### Limpeza e padronização dos dados")
    #     botao_silver = gr.Button("Transformar Bronze → Silver")
    #     silver_tabela = gr.DataFrame(label="Prévia dos dados Silver")
    #     botao_silver.click(transformar_para_silver, outputs=silver_tabela)

    # # Camada Gold
    # with gr.Tab("Camada Gold"):
    #     gr.Markdown("### Agregações e métricas finais")
    #     botao_gold = gr.Button("Transformar Silver → Gold")
    #     gold_tabela = gr.DataFrame(label="Tabela final Gold")
    #     botao_gold.click(transformar_para_gold, outputs=gold_tabela)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a652368d485fed3ef1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",